# [SOLUTION] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [1]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [2]:
load_dotenv()

True

## Play with Tavily

In [3]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [4]:
result = client.search("What's Nintendo?")

In [5]:
result

{'query': "What's Nintendo?",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Nintendo',
   'title': 'Nintendo',
   'content': '**Nintendo Co., Ltd.** is a Japanese multinational video game company headquartered in Kyoto. The history of Nintendo began when craftsman Fusajiro Yamauchi founded the company to produce handmade *hanafuda* playing cards. The company became internationally dominant in the 1980s after the arcade release of *Donkey Kong "Donkey Kong (1981 video game)")* (1981) and the Nintendo Entertainment System, which launched outside of Japan alongside *Super Mario Bros.* in 1985. Nintendo was founded as Nintendo Koppai on 23 September 1889 by craftsman Fusajiro Yamauchi in Shimogyō-ku, Kyoto, Japan, as an unincorporated establishment, to produce and distribute Japanese playing cards, or karuta (かるた; from Portuguese *carta*, \'card\'), most notably *hanafuda* (花札, \'flower cards\'). In 1953, Nintendo became t

## Define Web Search tool

In [7]:
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

In [8]:
tools = [web_search]

In [9]:
simple_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=("You are a helpful assistant"),
)

In [10]:
web_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
            "You are a web-aware assistant that can search for update information "
            "For each query, you will search the web for current information using " 
            "Tavily's AI-optimized search and provide a comprehensive answer\n"
            "Always cite your sources and explain any discrepancies found.\n"
            "Be particularly attentive to dates and time-sensitive information."
    ),
    tools=tools
)

In [11]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [12]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information on events that occurred after October 2023, including the winners of the 2025 Oscars. You may want to check the latest news or the official Oscars website for the most current information., tool_calls = None)


In [13]:
print(run1.get_final_state()["messages"][-1].content)

I'm sorry, but I don't have information on events that occurred after October 2023, including the winners of the 2025 Oscars. You may want to check the latest news or the official Oscars website for the most current information.


In [14]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information on events that occurred after October 2023, including the winners of the 2025 Oscars. You may want to check the latest news or the official Oscars website for the most current information., tool_calls = None)
 -> (role = user, content = What are the most recent developments in AI technology?, tool_calls = None)
 -> (role = assistant, content = As of my last update in October 2023, here are some notable developments in AI technology:

1. **Generative AI Advancements**: Generative models, particularly those like GPT-4 and DALL-E, have continued to improve in 

In [15]:
print(run2.get_final_state()["messages"][-1].content)

As of my last update in October 2023, here are some notable developments in AI technology:

1. **Generative AI Advancements**: Generative models, particularly those like GPT-4 and DALL-E, have continued to improve in generating text, images, and even music. These models have seen enhancements in their ability to understand context, generate coherent responses, and produce high-quality creative content.

2. **AI in Healthcare**: AI applications in healthcare have expanded, with advancements in predictive analytics for patient outcomes, AI-assisted diagnostics, and personalized medicine. Machine learning algorithms are increasingly used to analyze medical data, such as imaging and genetic information, to assist in treatment planning.

3. **Natural Language Processing (NLP)**: NLP technologies have advanced significantly, leading to more sophisticated chatbots and virtual assistants. These systems can understand and generate human-like text, leading to improvements in customer service, co

**Web Agent**

In [16]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_Bo1ZaQucqngJrMjJIAKYmn0K', function=Function(arguments='{"query":"2025 Oscar winner for International Feature Film","search_depth":"advanced"}', name=

In [17]:
print(run1.get_final_state()["messages"][-1].content)

The 2025 Oscar for Best International Feature Film was awarded to Brazil's "I'm Still Here." This film, directed by Walter Salles, marks Brazil's first win in this category and triumphed over other contenders, including "Emilia Pérez." 

The story is set in 1970s Rio de Janeiro during a time of military dictatorship and is based on true events. It explores themes of resilience and survival amidst challenging circumstances. The victory was somewhat unexpected as "Emilia Pérez" had been considered a strong contender leading up to the awards.

For more details, you can refer to sources like [NPR](https://www.npr.org/2025/03/02/nx-s1-5315313/oscars-2025-im-still-here-brazil-best-international-feature) and [Vanity Fair](https://www.vanityfair.com/hollywood/story/im-still-here-outlasts-emilia-perez-to-win-best-international-feature-at-oscars-2025?srsltid=AfmBOoocTv_hPAVVBAx66BTWgmxVsaiuPj8yf-EoCoyi1GUdmg4YjNOk).


In [18]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_Bo1ZaQucqngJrMjJIAKYmn0K', function=Function(arguments='{"query":"2025 Oscar winner for International Feature Film","search_depth":"advanced"}', name=

In [19]:
print(run2.get_final_state()["messages"][-1].content)

Recent developments in AI technology as of 2025 highlight significant advancements across various sectors:

1. **Generative AI and Creative Tools**: Companies like Adobe have introduced innovative AI tools within their Creative Cloud applications, enhancing features in Photoshop, Lightroom, and Premiere. This includes one-click tools for tasks such as compositing and masking, aimed at empowering creative professionals with greater efficiency and precision ([Adobe](https://news.adobe.com/news/2025/10/adobe-max-2025-creative-cloud)).

2. **Healthcare Innovations**: AI's integration into healthcare continues to expand, with the FDA approving numerous AI-enabled medical devices. AI models are now significantly improving cancer research methodologies and patient care ([Stanford HAI](https://hai.stanford.edu/ai-index/2025-ai-index-report)).

3. **Quantum Computing Breakthroughs**: Major tech companies, including Google, have reported advancements in quantum algorithms, which are now surpassi

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow